In [49]:
import estnltk
from estnltk.wordnet import wn
import pickle
import sqlite3

In [43]:
ssetC = wn.Synset
all_ssets_pkl = 'wordnet/data/all_synsets.pkl'
file_db       = 'wordnet/data/all_synsets.db'

In [3]:

synsetsDb = pickle.load(open(all_ssets_pkl, 'rb'))

In [297]:
relation_list =[]

In [301]:
def relation_test(cursor, synset, relation_str,end_vtx_id):
    
    if synset:
        if len(synset) > 1:
            #print(relation_str)
            for sset in synset:                
                tmp=str(sset)
                sset_name = tmp[9:-8]
                pos       = tmp[-7:-6]
                sense     = tmp[-5:-3]
                '''
                print(tmp)
                print(sset_name)
                print(pos)
                print(sense)
                '''
                cursor.execute('SELECT synset_id FROM synset_table WHERE synset_word = ? AND POS = ? AND sense = ?',(sset_name,pos,sense))
                row=cursor.fetchall()
                #print(row[0][0])
                start_vrtx.append(row[0][0])
                end_vrtx.append(end_vtx_id)
                relation_list.append(relation_str)
        if len(synset) == 1:
            tmp=str(synset)
            sset_name = tmp[10:-9]
            pos       = tmp[-8:-7]
            sense     = tmp[-6:-4]
            cursor.execute('SELECT synset_id FROM synset_table WHERE synset_word = ? AND POS = ? AND sense = ?',(sset_name,pos,sense))
            row=cursor.fetchall()
            '''
            print(relation_str)
            print(row)
            print(synset)
            print(pos)
            print(sense)
            '''
            start_vrtx.append(row[0][0])
            end_vrtx.append(end_vtx_id)
            relation_list.append(relation_str)

In [302]:

start_vrtx = []
end_vrtx   = []
relation   = []

def upload_hypernyms(file_db):

    conn = sqlite3.connect(file_db)
    cursor=conn.cursor()
    with conn:
        for i in range(len(synsetsDb)):
            #print("i: ", i)
            #print(synsetsDb[i])
            hypernym = ssetC.hypernyms(synsetsDb[i])
            hyponym  = ssetC.hyponyms(synsetsDb[i])
            holonym  = ssetC.holonyms(synsetsDb[i])
            meronym  = ssetC.meronyms(synsetsDb[i])
            relation_test(cursor,hypernym, "hypernym", i)
            relation_test(cursor,hyponym, "hyponym",  i)
            relation_test(cursor,holonym, "holonym", i)
            relation_test(cursor,meronym, "meronym", i)
                
                

In [339]:
for i in range(len(synsetsDb)):
    if ssetC.meronyms(synsetsDb[i]):
        print(ssetC.meronyms(synsetsDb[i]))
        print(i)

["Synset('immissioon.n.01')"]
115
["Synset('tootmisviis.n.01')"]
116
["Synset('sulg.n.04')"]
131
["Synset('pöörel.n.01')"]
143
["Synset('kalamari.n.01')", "Synset('poise.n.01')"]
149
["Synset('kabi.n.02')"]
157
["Synset('lauapõsed.n.01')", "Synset('parketrii.n.01')"]
164
["Synset('varjuk.n.01')"]
166
["Synset('vanker.n.04')"]
167
["Synset('voltnikerdis.n.01')"]
172
["Synset('sillutis.n.01')"]
173
["Synset('faas.n.04')", "Synset('allsüsteem.n.01')"]
177
["Synset('paneel.n.04')"]
179
["Synset('kaptaal.n.01')"]
183
["Synset('tagaiste.n.01')", "Synset('eesiste.n.01')", "Synset('kojamees.n.02')", "Synset('autouks.n.01')", "Synset('pärauks.n.01')"]
186


KeyboardInterrupt: 

In [303]:
upload_hypernyms(file_db)

KeyboardInterrupt: 

In [305]:
graph_db ='wordnet/data/relations_graph_db'

In [306]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS relations_graph(

                                        start_vertex INT NOT NULL,
                                        start_synset TEXT NOT NULL,
                                        end_synset TEXT NOT NULL,
                                        end_vertex INT NOT NULL,
                                        relation TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [331]:
def upload_graph_data(graph_db):

    sqlTables(graph_db)
    conn = create_connection(graph_db)
    cursor=conn.cursor()
    with conn:
        for i in range(len(relation_list)):
            start = start_vrtx[i]
            end   = end_vrtx[i]
            tmp = str(synsetsDb[start])
            start_sset = tmp[9:-8]
            tmp   = str(synsetsDb[end])
            end_sset = tmp[9:-8]
            cursor.execute(\
                "INSERT INTO relations_graph(start_vertex, start_synset, end_synset, end_vertex, relation) VALUES(?,?,?,?,?)"\
                           ,(start, start_sset, end_sset, end, relation_list[i]))
            conn.commit()

In [333]:
upload_graph_data(graph_db)

KeyboardInterrupt: 